In [1]:
import pandas as pd
import json
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [6]:
import json

# Function to encode overview text
def encode_overview(overview_text):
    # Your encoding function here

    input_ids = tokenizer.encode(overview_text, return_tensors='pt', max_length=768,padding=True, truncation=True)

    # Get the hidden states from the BERT model
    with torch.no_grad():
        outputs = model(input_ids)

    # Extract the last 2 hidden layers
    hidden_states = outputs.last_hidden_state[-2:]
    # Take the mean along the layer axis to get a single tensor [2, max_length, 768]
    mean_hidden_states = torch.mean(hidden_states, dim=1)

    # Take the mean along the token axis to get a fixed-size embedding for each overview [max_length, 768]
    overview_embedding = torch.mean(mean_hidden_states, dim=0)

    return overview_embedding

# Load the JSON file
with open('/content/sample_data/over_view.json', 'r') as json_file:
    data = json.load(json_file)

# Filter out key-value pairs where value is NaN or null
data_filtered = {key: value for key, value in data.items() if value is not None and not pd.isna(value)}

# Apply encode_overview function to each value
for key, value in data_filtered.items():
    encoded_embedding = encode_overview(value)
    data_filtered[key] = encoded_embedding.tolist()  # Convert tensor to list for JSON serialization

# Write the updated JSON object back to the file
with open('overview_embeddings.json', 'w') as json_file:
    json.dump(data_filtered, json_file)


In [9]:
## Just to Visualise how js

with open("/content/overview_embeddings.json", "r") as f:
    data = json.load(f)

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(data.items()), columns=["movies_id", "overviews"])

# Display the DataFrame
df


,movies_id,overviews
0,1,"[-0.25185492634773254, 0.23600180447101593, 0...."
1,2,"[-0.20528846979141235, 0.1400134414434433, 0.2..."
2,3,"[0.0502781942486763, -0.21238791942596436, 0.2..."
3,4,"[0.22480599582195282, 0.1149551123380661, 0.54..."
4,5,"[-0.04765688255429268, -0.08733636140823364, 0..."
...,...,...
2321,3947,"[-0.12200597673654556, 0.03812261298298836, 0...."
2322,3948,"[-0.2486516833305359, 0.1875748187303543, 0.24..."
2323,3949,"[0.15733693540096283, 0.2351396232843399, 0.27..."
2324,3950,"[-0.3209645748138428, 0.2864130735397339, -0.0..."


In [10]:
len(df['overviews'][0])

768

In [17]:
data = df.to_dict(orient='records')

# Write the list of dictionaries to a JSON file
with open('/content/overview_embeddings.json', 'w') as f:
    json.dump(data, f)